**This code imports necessary libraries and defines a function `connect_to_mt5` to establish a connection to the MetaTrader 5 (MT5) trading terminal. It initializes a connection by specifying the path to the terminal and retrieves account information (like account login, server, balance, and leverage) upon successful connection. The connection status is stored in the variable `mt5_connected`.**

In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Verbinding maken met MT5
def connect_to_mt5(path=None):
    """Maakt verbinding met MT5 terminal met foutafhandeling"""
    if not mt5.initialize(path=path):
        print(f"Initialize() mislukt, error code = {mt5.last_error()}")
        return False

    # Controleer of verbinding succesvol is
    if not mt5.terminal_info():
        print(f"Kon geen verbinding maken met terminal, error code = {mt5.last_error()}")
        return False

    # Print account informatie
    account_info = mt5.account_info()
    if account_info:
        print(f"Verbonden met account: {account_info.login}")
        print(f"Server: {account_info.server}")
        print(f"Balance: {account_info.balance}")
        print(f"Leverage: 1:{account_info.leverage}")
        return True
    return False

# Initialiseer verbinding
# Pas het pad aan naar jouw MT5 terminal als nodig
mt5_connected = connect_to_mt5(path="C:\\Program Files\\FTMO Global Markets MT5 Terminal\\terminal64.exe")

5. Tick Volume & Liquiditeit Analyse

In [ ]:
def analyze_liquidity(symbols, period=mt5.TIMEFRAME_H1, lookback=100):
    """
    Analyseert liquiditeit gebaseerd op tick volume

    Parameters:
    - symbols: Lijst met te analyseren symbolen
    - period: Timeframe voor analyse
    - lookback: Aantal candles om terug te kijken
    """
    from_date = datetime.now() - timedelta(days=lookback if period == mt5.TIMEFRAME_D1 else lookback//24)
    to_date = datetime.now()

    results = []

    for symbol in symbols:
        # Haal historische data op
        rates = mt5.copy_rates_range(symbol, period, from_date, to_date)
        if rates is None or len(rates) == 0:
            continue

        df = pd.DataFrame(rates)

        # Bereken liquiditeitsmetrieken
        avg_volume = df['tick_volume'].mean()
        max_volume = df['tick_volume'].max()
        min_volume = df['tick_volume'].min()

        # Bereken volatiliteit als secundaire metriek
        df['hl_range'] = (df['high'] - df['low'])
        avg_range = df['hl_range'].mean()

        # Contract details
        symbol_info = mt5.symbol_info(symbol)

        result = {
            'symbol': symbol,
            'avg_tick_volume': int(avg_volume),
            'max_tick_volume': int(max_volume),
            'min_tick_volume': int(min_volume),
            'avg_hl_range': float(avg_range),
            'timeframe': 'H1' if period == mt5.TIMEFRAME_H1 else 'D1',
            'spread': symbol_info.spread,
            'trade_mode': translate_trade_mode(symbol_info.trade_mode)
        }

        results.append(result)

    return pd.DataFrame(results)

def translate_trade_mode(trade_mode):
    """Vertaalt trade mode numerieke waarde naar leesbare tekst"""
    if trade_mode == mt5.SYMBOL_TRADE_MODE_DISABLED:
        return "Disabled"
    elif trade_mode == mt5.SYMBOL_TRADE_MODE_LONGONLY:
        return "Long Only"
    elif trade_mode == mt5.SYMBOL_TRADE_MODE_SHORTONLY:
        return "Short Only"
    elif trade_mode == mt5.SYMBOL_TRADE_MODE_CLOSEONLY:
        return "Close Only"
    elif trade_mode == mt5.SYMBOL_TRADE_MODE_FULL:
        return "Full"
    else:
        return "Unknown"

# Analyseer liquiditeit voor dezelfde symbolen
liquidity_df = analyze_liquidity(test_symbols)

print("\nLiquiditeitsanalyse:")
liquidity_df_sorted = liquidity_df.sort_values('avg_tick_volume', ascending=False)
print(liquidity_df_sorted[['symbol', 'avg_tick_volume', 'avg_hl_range', 'spread', 'trade_mode']].head(10))

6. Symboolspecifieke Trading Voorwaarde

In [ ]:
import MetaTrader5 as mt5  # Zorg ervoor dat deze import aanwezig is

def get_detailed_symbol_info(symbol):
    """Haalt gedetailleerde informatie op over een specifiek symbool"""
    symbol_info = mt5.symbol_info(symbol)
    if not symbol_info:
        print(f"Symbol '{symbol}' not found.")
        return None

    # Haal actuele tick info op
    tick = mt5.symbol_info_tick(symbol)

    # Bereid gedetailleerde informatie voor
    details = {
        "Basis Info": {
            "Naam": symbol_info.name,
            "Beschrijving": symbol_info.description,
            "ISIN": getattr(symbol_info, "isin", "Onbekend"),
            "Basis Valuta": symbol_info.currency_base,
            "Winst Valuta": symbol_info.currency_profit,
            "Categorie": symbol_info.path,
        },
        "Handelscondities": {
            "Contract Grootte": symbol_info.trade_contract_size,
            "Minimum Lot": symbol_info.volume_min,
            "Maximum Lot": symbol_info.volume_max,
            "Lot Stap": symbol_info.volume_step,
            "Swap Long": symbol_info.swap_long,
            "Swap Short": symbol_info.swap_short,
            "Swap Rollover 3-dagen": symbol_info.swap_rollover3days,
            "Margin Initieel": symbol_info.margin_initial,
            "Margin Onderhoud": symbol_info.margin_maintenance,
            "Hedge Margin": symbol_info.margin_hedged,
            "Margin Berekening Modus": getattr(symbol_info, "margin_mode", "Onbekend"),
        },
        "Prijsinformatie": {
            "Bid": tick.bid if tick else None,
            "Ask": tick.ask if tick else None,
            "Spread": (tick.ask - tick.bid) if tick else None,
            "Spread in punten": symbol_info.spread,
            "Spread is Variabel": bool(symbol_info.spread_float),
            "Stops Level": symbol_info.trade_stops_level,
            "Freeze Level": symbol_info.trade_freeze_level,
        },
        "Order Condities": {
            "Vul Modes": translate_filling_mode(getattr(symbol_info, "filling_mode", 0)),
            "Expiratie Modes": translate_expiration_mode(getattr(symbol_info, "expiration_mode", 0)),
            "Order Modes": translate_order_mode(getattr(symbol_info, "order_mode", 0)),
            "Trade Modes": translate_trade_mode(symbol_info.trade_mode),
        }
    }

    print("\nGedetailleerde informatie:")
    for category, info in details.items():
        print(f"\n{category}:")
        for key, value in info.items():
            print(f"  {key}: {value}")

    return details


def translate_filling_mode(filling_mode):
    """Vertaalt filling mode naar leesbare tekst"""
    modes = []
    if filling_mode & 1:  # SYMBOL_FILLING_FOK
        modes.append("Fill or Kill")
    if filling_mode & 2:  # SYMBOL_FILLING_IOC
        modes.append("Immediate or Cancel")
    if filling_mode & 4:  # SYMBOL_FILLING_RETURN
        modes.append("Return")
    return ", ".join(modes) if modes else "None"


def translate_expiration_mode(expiration_mode):
    """Vertaalt expiration mode naar leesbare tekst"""
    modes = []
    if expiration_mode & 1:  # SYMBOL_EXPIRATION_GTC
        modes.append("Good Till Cancelled")
    if expiration_mode & 2:  # SYMBOL_EXPIRATION_DAY
        modes.append("Day")
    if expiration_mode & 4:  # SYMBOL_EXPIRATION_SPECIFIED
        modes.append("Specified")
    if expiration_mode & 8:  # SYMBOL_EXPIRATION_SPECIFIED_DAY
        modes.append("Specified Day")
    return ", ".join(modes) if modes else "None"


def translate_order_mode(order_mode):
    """Vertaalt order mode naar leesbare tekst"""
    modes = []
    if order_mode & 1:  # SYMBOL_ORDER_MARKET
        modes.append("Market")
    if order_mode & 2:  # SYMBOL_ORDER_LIMIT
        modes.append("Limit")
    if order_mode & 4:  # SYMBOL_ORDER_STOP
        modes.append("Stop")
    if order_mode & 8:  # SYMBOL_ORDER_STOP_LIMIT
        modes.append("Stop Limit")
    if order_mode & 16:  # SYMBOL_ORDER_SL
        modes.append("Stop Loss")
    if order_mode & 32:  # SYMBOL_ORDER_TP
        modes.append("Take Profit")
    if order_mode & 64:  # SYMBOL_ORDER_CLOSURE
        modes.append("Closure")
    return ", ".join(modes) if modes else "None"


def translate_trade_mode(trade_mode):
    """Vertaalt trade mode naar leesbare tekst"""
    trade_modes = {
        0: "Disabled",
        1: "Long Only",
        2: "Short Only",
        3: "Close Only",
        4: "Full"
    }
    return trade_modes.get(trade_mode, "Unknown")


# Testvoorbeeld
if __name__ == "__main__":
    # Initialiseer MT5 connectie (zorg dat MT5 draait en je bent ingelogd)
    if not mt5.initialize():
        print("Initialisatie van MT5 mislukt")
    else:
        example_symbol = "EURUSD"
        get_detailed_symbol_info(example_symbol)
        mt5.shutdown()

6. Symboolspecifieke Trading Voorwaarden

In [11]:
def get_detailed_symbol_info(symbol):
    """Haalt gedetailleerde informatie op over een specifiek symbool"""
    symbol_info = mt5.symbol_info(symbol)
    if not symbol_info:
        return None

    # Haal actuele tick info op
    tick = mt5.symbol_info_tick(symbol)

    # Bereid gedetailleerde informatie voor
    details = {
        "Basis Info": {
            "Naam": symbol_info.name,
            "Beschrijving": symbol_info.description,
            "ISIN": symbol_info.isin,
            "Basis Valuta": symbol_info.currency_base,
            "Winst Valuta": symbol_info.currency_profit,
            "Categorie": symbol_info.path,
        },
        "Handelscondities": {
            "Contract Grootte": symbol_info.trade_contract_size,
            "Minimum Lot": symbol_info.volume_min,
            "Maximum Lot": symbol_info.volume_max,
            "Lot Stap": symbol_info.volume_step,
            "Swap Long": symbol_info.swap_long,
            "Swap Short": symbol_info.swap_short,
            "Swap Rollover 3-dagen": symbol_info.swap_rollover3days,
            "Margin Initieel": symbol_info.margin_initial,
            "Margin Onderhoud": symbol_info.margin_maintenance,
            "Hedge Margin": symbol_info.margin_hedged,
            "Margin Berekening Modus": symbol_info.margin_mode,
        },
        "Prijsinformatie": {
            "Bid": tick.bid if tick else None,
            "Ask": tick.ask if tick else None,
            "Spread": (tick.ask - tick.bid) if tick else None,
            "Spread in punten": symbol_info.spread,
            "Spread is Variabel": bool(symbol_info.spread_float),
            "Ticks Min": symbol_info.ticks_volume_min,
            "Ticks Max": symbol_info.ticks_volume_max,
            "Stops Level": symbol_info.stops_level,
            "Freeze Level": symbol_info.freeze_level,
        },
        "Order Condities": {
            "Vul Modes": translate_filling_mode(symbol_info.filling_mode),
            "Expiratie Modes": translate_expiration_mode(symbol_info.expiration_mode),
            "Order Modes": translate_order_mode(symbol_info.order_mode),
            "Trade Modes": translate_trade_mode(symbol_info.trade_mode),
        }
    }

    return details

def translate_filling_mode(filling_mode):
    """Vertaalt filling mode naar leesbare tekst"""
    modes = []
    if filling_mode & mt5.SYMBOL_FILLING_FOK:
        modes.append("Fill or Kill")
    if filling_mode & mt5.SYMBOL_FILLING_IOC:
        modes.append("Immediate or Cancel")
    if filling_mode & mt5.SYMBOL_FILLING_RETURN:
        modes.append("Return")
    return ", ".join(modes) if modes else "None"

def translate_expiration_mode(expiration_mode):
    """Vertaalt expiration mode naar leesbare tekst"""
    modes = []
    if expiration_mode & mt5.SYMBOL_EXPIRATION_GTC:
        modes.append("Good Till Cancelled")
    if expiration_mode & mt5.SYMBOL_EXPIRATION_DAY:
        modes.append("Day")
    if expiration_mode & mt5.SYMBOL_EXPIRATION_SPECIFIED:
        modes.append("Specified")
    if expiration_mode & mt5.SYMBOL_EXPIRATION_SPECIFIED_DAY:
        modes.append("Specified Day")
    return ", ".join(modes) if modes else "None"

def translate_order_mode(order_mode):
    """Vertaalt order mode naar leesbare tekst"""
    modes = []
    if order_mode & mt5.SYMBOL_ORDER_MARKET:
        modes.append("Market")
    if order_mode & mt5.SYMBOL_ORDER_LIMIT:
        modes.append("Limit")
    if order_mode & mt5.SYMBOL_ORDER_STOP:
        modes.append("Stop")
    if order_mode & mt5.SYMBOL_ORDER_STOP_LIMIT:
        modes.append("Stop Limit")
    if order_mode & mt5.SYMBOL_ORDER_SL:
        modes.append("Stop Loss")
    if order_mode & mt5.SYMBOL_ORDER_TP:
        modes.append("Take Profit")
    if order_mode & mt5.SYMBOL_ORDER_CLOSURE:
        modes.append("Closure")
    return ", ".join(modes) if modes else "None"

# Kies een voorbeeldsymbool, bijv. EURUSD
example_symbol = "EURUSD"
if example_symbol in available_symbols['name'].values:
    detailed_info = get_detailed_symbol_info(example_symbol)

    print(f"\nGedetailleerde Info voor {example_symbol}:")
    for category, info in detailed_info.items():
        print(f"\n{category}:")
        for key, value in info.items():
            print(f"  {key}: {value}")

NameError: name 'available_symbols' is not defined

markturen visualisatie


In [ ]:
def visualize_market_hours(sessions_df):
    """Visualiseert markttijden voor verschillende symbolen"""
    # Beperkt tot weekdagen
    days = ['mon', 'tue', 'wed', 'thu', 'fri']

    # Maak plot
    fig, ax = plt.subplots(figsize=(12, 8))

    # Voor elk symbool
    for i, row in sessions_df.iterrows():
        symbol = row['symbol']

        # Voor elke dag
        for day_idx, day in enumerate(days):
            open_time = row[f'{day}_open']
            close_time = row[f'{day}_close']

            # Skip als gesloten
            if open_time == "Closed" or close_time == "Closed":
                continue

            # Converteer tijd naar uren (voor plot)
            open_hour = convert_time_to_float(open_time)
            close_hour = convert_time_to_float(close_time)

            # Plot handelssessie
            ax.barh(y=i, width=close_hour-open_hour, left=open_hour,
                   height=0.5, color=f'C{day_idx}', alpha=0.7,
                   label=day.capitalize() if i == 0 else "")

    # Voeg labels toe
    ax.set_yticks(range(len(sessions_df)))
    ax.set_yticklabels(sessions_df['symbol'])
    ax.set_xlabel('Tijd (GMT)')
    ax.set_xlim(0, 24)
    ax.set_xticks(range(0, 25, 4))
    ax.set_xticklabels([f"{h:02d}:00" for h in range(0, 25, 4)])
    ax.set_title('Handelstijden per Symbool')
    ax.grid(True, axis='x', linestyle='--', alpha=0.7)

    # Voeg legenda toe voor dagen
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, labels, title="Dag", loc='upper center',
             bbox_to_anchor=(0.5, -0.15), ncol=5)

    plt.tight_layout()
    return fig

def convert_time_to_float(time_str):
    """Converteert tijd string naar float voor plotting"""
    if time_str == "Closed":
        return 0

    hours, minutes = map(int, time_str.split(':'))
    return hours + minutes/60

# Voeg een controle toe voor sessions_info
if 'sessions_info' in locals():
    if not sessions_info.empty:
        market_hours_fig = visualize_market_hours(sessions_info)
        plt.show()
else:
    print("Error: De 'sessions_info' dataset is niet gedefinieerd.")